In [1]:
import os
import json
import copy

# List result Files
experiment_files = [a for a in os.listdir("./out") if ".json" in a]

# Load result Files
all_experiments = []
for filename in experiment_files:
    with open("./out/{}".format(filename), "r") as f:
        all_experiments.append(json.load(f))

# Aglomerate similar results
unique_experiment_names = list(dict.fromkeys([a["model"] for a in all_experiments]))
print(unique_experiment_names)

sorted_experiments = {}
for e in unique_experiment_names:
    sorted_experiments[e] = []

for e in all_experiments:
    sorted_experiments[e['model']].append(e)

['assine2022b_33', 'assine2022b_44', 'assine2022b_14', 'matsubara2022_1', 'lee2021_layer_3', 'matsubara2022_3', 'matsubara2022_2', 'assine2022a_50', 'assine2022b_24', 'assine2022a_75', 'matsubara2022_4', 'lee2021_layer_10', 'assine2022b_22', 'lee2021_layer_7', 'lee2021_layer_5', 'assine2022a_100', 'assine2022b_34', 'dummy', 'assine2022a_25', 'matsubara2022_5', 'assine2022b_11']


In [2]:
baseline = sorted_experiments['dummy']

total_energy = sum([b['joules1'] for b in baseline])
total_seconds = sum([b['seconds'] for b in baseline])
baseline_power = total_energy/total_seconds
print(baseline_power)

2.87708821265


In [ ]:
aggregated_experiments = []

for exp_name, exp_list in sorted_experiments.items():
    joules = sum([e['joules1'] for e in exp_list])
    seconds = sum([e['seconds'] for e in exp_list])
    corrected_joules = joules - baseline_power*seconds
    num_images = sum([e['num_images'] for e in exp_list])
    aggregated_experiments.append({
        "mAP": exp_list[0]['map'],
        "inference_time": 1e3*seconds / num_images,
        "KB": exp_list[0]['bw']/1e3,
        "joules": joules,
        "seconds": seconds,
        "corrected_joules": joules - baseline_power*seconds,
        "corrected_watts": (joules - baseline_power*seconds)/seconds,
        "corrected_joules_per_image": corrected_joules / num_images,
        "num_images": num_images,
        "model": exp_name,
        "GMAC": exp_list[0]['macs']/1e9,
        "params(K)": exp_list[0]['params']/1e3,
    })
    
with open("test.json", "w+") as f:
    json.dump(aggregated_experiments, f)

In [ ]:
import pandas as pd
df = pd.read_json('test.json')
df = df.set_index("model")
df = df.sort_index(ascending=True)
df = df.drop(["joules", "seconds","corrected_joules", "num_images"], axis=1)

rename_map = {
    "corrected_joules_per_image": "Joules",
    "corrected_watts": "Watts",
    "inference_time": "ms",
}

df = df.rename(rename_map, axis=1)

df['Watts'] = df['Watts'].round(2)
df['Joules'] = df['Joules'].round(2)
df['ms'] = df['ms'].round(0).astype(int)
df['GMAC'] = df['GMAC'].round(3)
df['KB'] = df['KB'].round(1)
df['mAP'] = df['mAP'].round(1)
df['params(K)'] = df['params(K)'].round(1)
print(df)

df_back = copy.copy(df)


a = df.reindex(["assine2022a_25", "assine2022a_75", "assine2022a_100"])

In [ ]:
print(df.to_latex()) 

In [ ]:
import matplotlib
font = {'family' : 'normal', 'size'   : 22}
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt

df = copy.copy(df_back)
###########################################################################
def plot(ax, name, color='blue', configurable=False):
    a = df[df.index.str.contains(name)]
    bw = a['KB'].values
    latency = a['ms'].values
    mAP = a['mAP'].values
    
    z = 2.5*(mAP**2)
    z_label = ["{:2.1f}".format(m) for m in mAP]
    if configurable:
         ax.scatter(latency, bw, s=z, marker='o', color=color, alpha=0.25, 
                    label=name, hatch='||')#, hatch='||') #, edgecolor=color, linewidth=4
    else:
        ax.scatter(latency, bw, s=z, marker='o', color=color, alpha=0.3,
                   edgecolor='black', linewidth=2, label=name)
    for i in range(len(mAP)):
        ax.annotate(z_label[i], ( latency[i], bw[i]), ha='center', va='center', fontsize=18, color='black')



fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
ax.set_xlabel('compute time (ms)')
ax.set_ylabel('bandwidth (KB)')
ax.set_xscale('log')
ax.set_yscale('log')

f = "literature_comparison/literature.csv"
df.columns = df.columns.str.replace(' ', '')
plot(ax, "lee2021", color='cyan')
plot(ax, "assine2022b", color='red', configurable=True)
plot(ax, "assine2022a", color="#A8127D", configurable=True)
plot(ax, "matsubara2021", color='blue')

# plot("assine", color='black',  configurable=True)

# bb = t.get_bbox_patch()
# bb.set_boxstyle("rarrow", pad=0.6)



# bbox_props = dict(boxstyle="rarrow", fc=(0.7, 0.1, 0.1), ec="b", lw=0)
bbox_props = dict(boxstyle="rarrow", fc="#0b5394", ec="b", lw=0)
t = ax.text(580, 1.1e1, "Inverse Tradeoff", ha="center", va="center", rotation=-60,
            size=20,
            bbox=bbox_props)

# bbox_props = dict(boxstyle="larrow", fc=(0.1, 0.1, 0.7), ec="b", lw=0)
# t = ax.text(210, 9, "Direct Tradeoff", ha="center", va="center", rotation=40,
#             size=18,
#             bbox=bbox_props)
bbox_props = dict(boxstyle="larrow", fc="#a64d79", ec="b", lw=0)
t = ax.text(250, 9, "Direct Tradeoff", ha="center", va="center", rotation=35,
            size=20,
            bbox=bbox_props)

legend_properties = {'weight':'bold', 'size': 16}
legend_properties = {'size': 25}
lgnd = ax.legend(fontsize=15,  prop=legend_properties, loc='upper left',)#bbox_to_anchor=(0.1, 0.6),
lgnd.legendHandles[0]._sizes = [1e3]
lgnd.legendHandles[1]._sizes = [1e3]
lgnd.legendHandles[2]._sizes = [1e3]
lgnd.legendHandles[3]._sizes = [1e3]

plt.savefig(os.path.join("./out", "graph_sota.pdf"))
plt.show()
